In [106]:
import chess
import chess.pgn
import os
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import time
import sys


In [ ]:
board = chess.Board()
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [ ]:
pgn = open("./data/KingBase2018-pgn/KingBase2018-E60-E99.pgn")

In [123]:
#counting number of games in one dataset:
#Simple slow code:

# num_games = 0
# while True:
#     game = chess.pgn.read_game(pgn)
#     if game == None:
#         break
#     else:
#         num_games += 1
#         print(num_games)
# print(num_games)

#fast code:
num_games = 0


with open("./data/KingBase2018-pgn/KingBase2018-E60-E99.pgn", 'r') as f2:
    for line in f2:
        #print(line)
        if line.startswith("[Event"):
            num_games += 1

with open("./data/KingBase2018-pgn/KingBase2018-E60-E99.pgn", "r", errors="ignore") as f:
    data = f.read()
    print(f' type of png data: {type(data)}') 
    print(f'size of png data: {sys.getsizeof(data)}')
    

print(f"number of games: {num_games}")

 type of png data: <class 'str'>
size of png data: 87095098
number of games: 237619


In [ ]:
# Create a ThreadPoolExecutor without specifying max_workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    print(f"Default max_workers: {executor._max_workers}")

def task(n):
    print(f'Task {n} starting')
    time.sleep(2)
    print(f'Task {n} done')
    return n

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(task, i) for i in range(8)]
    for future in concurrent.futures.as_completed(futures):
        print(f"Result : {future.result}")

Default max_workers: 8
Task 0 starting
Task 1 starting
Task 2 starting
Task 3 starting
Task 4 starting
Task 5 starting
Task 6 starting
Task 7 starting


Task 0 done
Task 2 done
Task 3 done
Task 4 done
Result : <bound method Future.result of <Future at 0x239c78c76d0 state=finished returned int>>
Result : <bound method Future.result of <Future at 0x239c79c8f50 state=finished returned int>>
Result : <bound method Future.result of <Future at 0x239c7b905d0 state=finished returned int>>
Result : <bound method Future.result of <Future at 0x239c7b904d0 state=finished returned int>>
Task 5 done
Result : <bound method Future.result of <Future at 0x239c7b90550 state=finished returned int>>
Task 1 done
Result : <bound method Future.result of <Future at 0x239c7b90650 state=finished returned int>>
Task 6 done
Result : <bound method Future.result of <Future at 0x239c7b906d0 state=finished returned int>>
Task 7 done
Result : <bound method Future.result of <Future at 0x239c7b907d0 state=finished returned int>>


In [ ]:
# Create Minimal ThreadPoolExecutor 



In [ ]:


def read_pgn_in_chunks(file_path, chunk_size=2048):  # Adjust here
    with open(file_path, 'r') as file:
        while True:
            chunk = list(file.readline() for _ in range(chunk_size))
            if not chunk:
                break
            yield chunk

file_path = './data/KingBase2018-pgn/KingBase2018-E60-E99.pgn'
#chunk = read_pgn_in_chunks(file_path, chunk_size=2048)
#print(chunk)

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(read_pgn_in_chunks, file_path, chunk_size=2048)
    print(future.result())

<generator object read_pgn_in_chunks at 0x00000239EEE07560>


In [ ]:
total_workers = 0
total_size_of_single_pgn = 0

with concurrent.futures.ThreadPoolExecutor() as executor:
    print(f"Default max_workers: {executor._max_workers}")
    total_workers = executor._max_workers

with open("./data/KingBase2018-pgn/KingBase2018-E60-E99.pgn", 'r') as f2:
    data = f2.read()
    #print(type(data)) #output : <class 'str'>
    print(sys.getsizeof(data))
    total_size_of_single_pgn = sys.getsizeof(data) 



In [ ]:
import chess.pgn
from concurrent.futures import ThreadPoolExecutor

def count_games_in_chunk(file_chunk):
    num_games = 0
    for line in file_chunk:
        game = chess.pgn.read_game(line)
        if game is not None:
            num_games += 1
    return num_games

def read_pgn_in_chunks(file_path, chunk_size=1024):
    with open(file_path, 'r') as file:
        while True:
            chunk = list(file.readline() for _ in range(chunk_size))
            if not chunk:
                break
            yield chunk

def count_total_games(file_path):
    total_games = 0
    with ThreadPoolExecutor() as executor:
        futures = []
        for file_chunk in read_pgn_in_chunks(file_path):
            futures.append(executor.submit(count_games_in_chunk, file_chunk))
        
        for future in futures:
            total_games += future.result()
    
    return total_games

# Example usage
file_path = './data/KingBase2018-pgn/KingBase2018-E60-E99.pgn'
#total_games = count_total_games(file_path)
print(f"Total number of games: {total_games}")


NameError: name 'total_games' is not defined

In [ ]:
#opening all datasets:


In [ ]:
class State():
    def __init__(self):
        self.board = chess.Board()

    def edges(self):
        return list(self.board.legal_moves)

    def value(self):
        
        return 1

    

    

In [ ]:
state = State()
print(state.edges())

[Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('b1a3'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('a2a3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e2e4'), Move.from_uci('d2d4'), Move.from_uci('c2c4'), Move.from_uci('b2b4'), Move.from_uci('a2a4')]
